In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 503, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 503 (delta 163), reused 132 (delta 118), pack-reused 297
Receiving objects: 100% (503/503), 1.49 MiB | 6.90 MiB/s, done.
Resolving deltas: 100% (290/290), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1325 (delta 84), reused 89 (delta 41), pack-reused 1178
Receiving objects: 100% (1325/1325), 9.57 MiB | 11.37 MiB/s, done.
Resolving deltas: 100% (688/688), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 

In [ ]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq

!pip install --editable ./

%cd ..

     |████████████████████████████████| 895 kB 2.3 MB/s 
     |████████████████████████████████| 90 kB 8.4 MB/s 
     |████████████████████████████████| 124 kB 45.1 MB/s 
     |████████████████████████████████| 40 kB 5.3 MB/s 
     |████████████████████████████████| 2.8 MB 41.3 MB/s 
  Created wheel for mosestokenizer: filename=mosestokenizer-1.1.0-py3-none-any.whl size=49117 sha256=2164036ade47d1d00b80c42409989cb344c050043284c34151696831006f5d7c
  Stored in directory: /root/.cache/pip/wheels/a7/31/94/fef279382208e85a65c1a7f5c4d0020115477b0af74f296b57
  Created wheel for toolwrapper: filename=toolwrapper-2.1.0-py3-none-any.whl size=3354 sha256=139195374012e4c3b07ad984a58f20e3a4e5c65fdd5af5dc32e18ed8da8a1f5b
  Stored in directory: /root/.cache/pip/wheels/c5/4f/33/54741ffe08e38ececb1d28068a153729b4fe820bafa0a0691f
  Created wheel for uctools: filename=uctools-1.3.0-py3-none-any.whl size=6163 sha256=6b259f26e24ad557ce9ee523d6cb25333f27a6f81e1261dcc88a5b610c65202e
  Stored in directory: /r

In [ ]:
# this step is only required if you are running the code on colab
# restart the runtime after running prev cell (to update). See this -> https://stackoverflow.com/questions/57838013/modulenotfounderror-after-successful-pip-install-in-google-colaboratory

# this import will not work without restarting runtime
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [ ]:
# download the indictrans model


# downloading the indic-en model
!wget https://storage.googleapis.com/samanantar-public/V0.2/models/indic-en.zip
!unzip indic-en.zip

# downloading the en-indic model
# !wget https://storage.googleapis.com/samanantar-public/V0.2/models/en-indic.zip
# !unzip en-indic.zip

# # downloading the indic-indic model
# !wget https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
# !unzip m2m.zip

%cd indicTrans

--2021-10-01 10:53:16--  https://storage.googleapis.com/samanantar-public/V0.2/models/indic-en.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.191.128, 173.194.74.128, 173.194.192.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.191.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4551079075 (4.2G) [application/zip]
Saving to: ‘indic-en.zip’

indic-en.zip        100%[===================>]   4.24G  38.6MB/s    in 78s     

2021-10-01 10:54:34 (55.7 MB/s) - ‘indic-en.zip’ saved [4551079075/4551079075]

Archive:  indic-en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/dict.TGT.txt  
  inflating: indic-en/final_bin/dict.SRC.txt  
   creating: indic-en/model/
  inf

In [ ]:
from indicTrans.inference.engine import Model
indic2en_model = Model(expdir='../indic-en')

Initializing vocab and bpe
Initializing model for translation


2021-10-01 10:57:27 | INFO | fairseq.tasks.translation | [SRC] dictionary: 35864 types
2021-10-01 10:57:27 | INFO | fairseq.tasks.translation | [TGT] dictionary: 32088 types


In [ ]:
ta_paragraph ="""झोपड़े के द्वार पर बाप और बेटा दोनों एक बुझे हुए अलाव के सामने चुपचाप बैठे हुए हैं और अंदर बेटे की जवान बीबी बुधिया प्रसव वेदना में पछाड़ खा रही थी. रह रहकर उसके मुंह से ऐसी दिल हिला देने वाली आवाज़ निकलती थी, कि दोनों कलेजा थाम लेते थे. जाड़ों की रात थी, प्रकृति सन्नाटे में डूबी हुई, सारा गांव अंधकार में लय हो गया था.चमारों का कुनबा था और सारे गांव में बदनाम. घीसू एक दिन काम करता तो तीन दिन आराम करता. माधव इतना काम चोर था कि आध घंटे काम करता तो घंटे भर चिलम पीता. इसलिए उन्हें कहीं मजदूरी नहीं मिलती थी. घर में मुठ्ठी भर भी अनाज मौजूद हो, तो उनके लिए काम करने की कसम थी. जब दो चार फाके हो जाते तो घीसू पेड़ पर चढक़र लकडिय़ां तोड़ लाता और माधव बाजार से बेच लाता और जब तक वह पैसे रहते, दोनों इधर उधर मारे मारे फिरते.

गांव में काम की कमी न थी. किसानों का गांव था, मेहनती आदमी के लिए पचास काम थे. मगर इन दोनों को उसी वक्त बुलाते, जब दो आदमियों से एक का काम पाकर भी सन्तोष कर लेने के सिवा और कोई चारा न होता. अगर दोनो साधु होते, तो उन्हें सन्तोष और धैर्य के लिए, संयम और नियम की बिलकुल जरूरत न होती. यह तो इनकी प्रकृति थी. विचित्र जीवन था इनका! घर में मिट्टी के दो चार बर्तन के सिवा कोई सम्पत्ति नहीं. फटे चीथड़ों से अपनी नग्नता को ढांके हुए जिये जाते थे. संसार की चिन्ताओं से मुक्त कर्ज से लदे हुए. गालियां भी खाते, मार भी खाते, मगर कोई गम नहीं. दीन इतने कि वसूली की बिलकुल आशा न रहने पर भी लोग इन्हें कुछ न कुछ कर्ज दे देते थे. मटर, आलू की फसल में दूसरों के खेतों से मटर या आलू उखाड़ लाते और भून भानकर खा लेते या दस पांच ऊख उखाड़ लाते और रात को चूसते.

घीसू ने इसी आकाश वृत्ति से साठ साल की उम्र काट दी और माधव भी सपूत बेटे की तरह बाप ही के पद चिह्नों पर चल रहा था, बल्कि उसका नाम और भी उजागर कर रहा था. इस वक्त भी दोनों अलाव के सामने बैठकर आलू भून रहे थे, जो कि किसी खेत से खोद लाये थे. घीसू की स्त्री का तो बहुत दिन हुए, देहान्त हो गया था. माधव का ब्याह पिछले साल हुआ था. जब से यह औरत आयी थी, उसने इस खानदान में व्यवस्था की नींव डाली थी और इन दोनों बे गैरतों का दोजख भरती रहती थी. जब से वह आयी, यह दोनों और भी आरामतलब हो गये थे. बल्कि कुछ अकडऩे भी लगे थे. कोई कार्य करने को बुलाता, तो निब्र्याज भाव से दुगुनी मजदूरी मांगते. वही औरत आज प्रसव वेदना से मर रही थी और यह दोनों इसी इन्तजार में थे कि वह मर जाए, तो आराम से सोयें.

घीसू ने आलू निकालकर छीलते हुए कहा. जाकर देख तो, क्या दशा है उसकी? चुड़ैल का फिसाद होगा, और क्या? यहां तो ओझा भी एक रुपया मांगता है!

माधव को भय था, कि वह कोठरी में गया, तो घीसू आलुओं का बड़ा भाग साफ कर देगा. बोला. मुझे वहां जाते डर लगता है.

‘डर किस बात का है, मैं तो यहां हूं ही.’

‘तो तुम्हीं जाकर देखो न?’

‘मेरी औरत जब मरी थी, तो मैं तीन दिन तक उसके पास से हिला तक नहीं; और फिर मुझसे लजाएगी कि नहीं? जिसका कभी मुंह नहीं देखा, आज उसका उघड़ा हुआ बदन देखूं! उसे तन की सुध भी तो न होगी? मुझे देख लेगी तो खुलकर हाथ पांव भी न पटक सकेगी!’

‘मैं सोचता हूं कोई बाल बच्चा हुआ, तो क्या होगा? सोंठ, गुड़, तेल, कुछ भी तो नहीं है घर में!’

‘सब कुछ आ जाएगा. भगवान दें तो! जो लोग अभी एक पैसा नहीं दे रहे हैं, वे ही कल बुलाकर रुपये देंगे. मेरे नौ लड़के हुए, घर में कभी कुछ न था; मगर भगवान ने किसी न किसी तरह बेड़ा पार ही लगाया.’

जिस समाज में रात दिन मेहनत करने वालों की हालत उनकी हालत से कुछ बहुत अच्छी न थी, और किसानों के मुकाबले में वे लोग, जो किसानों की दुर्बलताओं से लाभ उठाना जानते थे, कहीं ज्यादा सम्पन्न थे, वहां इस तरह की मनोवृत्ति का पैदा हो जाना कोई अचरज की बात न थी. हम तो कहेंगे, घीसू किसानों से कहीं ज्यादा विचारवान था और किसानों के विचार शून्य समूह में शामिल होने के बदले बैठकबाजों की कुत्सित मण्डली में जा मिला था. हां, उसमें यह शक्ति न थी, कि बैठकबाजों के नियम और नीति का पालन करता. इसलिए जहां उसकी मण्डली के और लोग गांव के सरगना और मुखिया बने हुए थे, उस पर सारा गांव उंगली उठाता था. फिर भी उसे यह तसकीन तो थी ही कि अगर वह फटेहाल है तो कम से कम उसे किसानों की सी जी तोड़ मेहनत तो नहीं करनी पड़ती, और उसकी सरलता और निरीहता से दूसरे लोग बेजा फायदा तो नहीं उठाते!

दोनों आलू निकाल निकालकर जलते जलते खाने लगे. कल से कुछ नहीं खाया था. इतना सब्र न था कि ठण्डा हो जाने दें. कई बार दोनों की जबानें जल गयीं. छिल जाने पर आलू का बाहरी हिस्सा जबान, हलक और तालू को जला देता था और उस अंगारे को मुंह में रखने से ज्यादा खैरियत इसी में थी कि वह अंदर पहुंच जाए. वहां उसे ठण्डा करने के लिए काफी सामान थे. इसलिए दोनों जल्द जल्द निगल जाते. हालांकि इस कोशिश में उनकी आंखों से आंसू निकल आते.

घीसू को उस वक्त ठाकुर की बरात याद आयी, जिसमें बीस साल पहले वह गया था. उस दावत में उसे जो तृप्ति मिली थी, वह उसके जीवन में एक याद रखने लायक बात थी, और आज भी उसकी याद ताजी थी, बोला वह भोज नहीं भूलता. तब से फिर उस तरह का खाना और भरपेट नहीं मिला. लडक़ी वालों ने सबको भर पेट पूडिय़ां खिलाई थीं, सबको! छोटे बड़े सबने पूडिय़ां खायीं और असली घी की! चटनी, रायता, तीन तरह के सूखे साग, एक रसेदार तरकारी, दही, चटनी, मिठाई, अब क्या बताऊं कि उस भोज में क्या स्वाद मिला, कोई रोक टोक नहीं थी, जो चीज चाहो, मांगो, जितना चाहो, खाओ. लोगों ने ऐसा खाया, ऐसा खाया, कि किसी से पानी न पिया गया. मगर परोसने वाले हैं कि पत्तल में गर्म गर्म, गोल गोल सुवासित कचौडिय़ां डाल देते हैं. मना करते हैं कि नहीं चाहिए, पत्तल पर हाथ रोके हुए हैं, मगर वह हैं कि दिये जाते हैं. और जब सबने मुंह धो लिया, तो पान इलायची भी मिली. मगर मुझे पान लेने की कहां सुध थी? खड़ा हुआ न जाता था. चटपट जाकर अपने कम्बल पर लेट गया. ऐसा दिल दरियाव था वह ठाकुर!

माधव ने इन पदार्थों का मन ही मन मजा लेते हुए कहा अब हमें कोई ऐसा भोज नहीं खिलाता.

‘अब कोई क्या खिलाएगा? वह जमाना दूसरा था. अब तो सबको किफायत सूझती है. सादी ब्याह में मत खर्च करो, क्रिया कर्म में मत खर्च करो. पूछो, गरीबों का माल बटोर बटोरकर कहां रखोगे? बटोरने में तो कमी नहीं है. हां, खर्च में किफायत सूझती है!’

‘तुमने एक बीस पूरियां खायी होंगी?’

‘बीस से ज्यादा खायी थीं!’

‘मैं पचास खा जाता!’

‘पचास से कम मैंने न खायी होंगी. अच्छा पका था. तू तो मेरा आधा भी नहीं है.’

आलू खाकर दोनों ने पानी पिया और वहीं अलाव के सामने अपनी धोतियां ओढ़कर पांव पेट में डाले सो रहे. जैसे दो बड़े बड़े अजगर गेंडुलिया मारे पड़े हों.

और बुधिया अभी तक कराह रही थी.

सबेरे माधव ने कोठरी में जाकर देखा, तो उसकी स्त्री ठण्डी हो गयी थी. उसके मुंह पर मक्खियां भिनक रही थीं. पथराई हुई आंखें ऊपर टंगी हुई थीं. सारी देह धूल से लथपथ हो रही थी. उसके पेट में बच्चा मर गया था.

माधव भागा हुआ घीसू के पास आया. फिर दोनों जोर जोर से हाय हाय करने और छाती पीटने लगे. पड़ोस वालों ने यह रोना धोना सुना, तो दौड़े हुए आये और पुरानी मर्यादा के अनुसार इन अभागों को समझाने लगे.

मगर ज्यादा रोने पीटने का अवसर न था. कफ़न की और लकड़ी की फिक्र करनी थी. घर में तो पैसा इस तरह गायब था, जैसे चील के घोंसले में मांस?

बाप बेटे रोते हुए गांव के जमींदार के पास गये. वह इन दोनों की सूरत से नफ़रत करते थे. कई बार इन्हें अपने हाथों से पीट चुके थे. चोरी करने के लिए, वादे पर काम पर न आने के लिए. पूछा क्या है बे घिसुआ, रोता क्यों है? अब तो तू कहीं दिखलाई भी नहीं देता! मालूम होता है, इस गांव में रहना नहीं चाहता.

घीसू ने जमीन पर सिर रखकर आंखों में आंसू भरे हुए कहा सरकार! बड़ी विपत्ति में हूं. माधव की घरवाली रात को गुजर गयी. रात भर तड़पती रही सरकार! हम दोनों उसके सिरहाने बैठे रहे. दवा दारू जो कुछ हो सका, सब कुछ किया, मुदा वह हमें दगा दे गयी. अब कोई एक रोटी देने वाला भी न रहा मालिक! तबाह हो गये. घर उजड़ गया. आपका गुलाम हूं, अब आपके सिवा कौन उसकी मिट्टी पार लगाएगा. हमारे हाथ में तो जो कुछ था, वह सब तो दवा दारू में उठ गया. सरकार ही की दया होगी, तो उसकी मिट्टी उठेगी. आपके सिवा किसके द्वार पर जाऊं.

जमींदार साहब दयालु थे. मगर घीसू पर दया करना काले कम्बल पर रंग चढ़ाना था. जी में तो आया, कह दें, चल, दूर हो यहां से. यों तो बुलाने से भी नहीं आता, आज जब गरज पड़ी तो आकर खुशामद कर रहा है. हरामखोर कहीं का, बदमाश! लेकिन यह क्रोध या दण्ड देने का अवसर न था. जी में कुढ़ते हुए दो रुपये निकालकर फेंक दिए. मगर सान्त्वना का एक शब्द भी मुंह से न निकला. उसकी तरफ ताका तक नहीं. जैसे सिर का बोझ उतारा हो.

जब जमींदार साहब ने दो रुपये दिये, तो गांव के बनिये महाजनों को इनकार का साहस कैसे होता? घीसू जमींदार के नाम का ढिंढोरा भी पीटना जानता था. किसी ने दो आने दिये, किसी ने चारे आने. एक घंटे में घीसू के पास पांच रुपये की अच्छी रकम जमा हो गयी. कहीं से अनाज मिल गया, कहीं से लकड़ी. और दोपहर को घीसू और माधव बाज़ार से कफ़न लाने चले. इधर लोग बांस वांस काटने लगे.

गांव की नर्मदिल स्त्रियां आ आकर लाश देखती थीं और उसकी बेकसी पर दो बूंद आंसू गिराकर चली जाती थीं.

बाज़ार में पहुंचकर घीसू बोला लकड़ी तो उसे जलाने भर को मिल गयी है, क्यों माधव!

माधव बोला हां, लकड़ी तो बहुत है, अब कफ़न चाहिए.

‘तो चलो, कोई हलका सा कफ़न ले लें.’

‘हां, और क्या! लाश उठते उठते रात हो जाएगी. रात को कफ़न कौन देखता है?’

‘कैसा बुरा रिवाज है कि जिसे जीते जी तन ढांकने को चीथड़ा भी न मिले, उसे मरने पर नया कफ़न चाहिए.’

‘कफ़न लाश के साथ जल ही तो जाता है.’

‘और क्या रखा रहता है? यही पांच रुपये पहले मिलते, तो कुछ दवा दारू कर लेते.’

दोनों एक दूसरे के मन की बात ताड़ रहे थे. बाजार में इधर उधर घूमते रहे. कभी इस बजाज की दूकान पर गये, कभी उसकी दूकान पर! तरह तरह के कपड़े, रेशमी और सूती देखे, मगर कुछ जंचा नहीं. यहां तक कि शाम हो गयी. तब दोनों न जाने किस दैवी प्रेरणा से एक मधुशाला के सामने जा पहुंचे. और जैसे किसी पूर्व निश्चित व्यवस्था से अंदर चले गये. वहां जरा देर तक दोनों असमंजस में खड़े रहे. फिर घीसू ने गद्दी के सामने जाकर कहा साहूजी, एक बोतल हमें भी देना.

उसके बाद कुछ चिखौना आया, तली हुई मछली आयी और दोनों बरामदे में बैठकर शान्तिपूर्वक पीने लगे.

कई कुज्जियां ताबड़तोड़ पीने के बाद दोनों सरूर में आ गये.

घीसू बोला कफ़न लगाने से क्या मिलता? आखिर जल ही तो जाता. कुछ बहू के साथ तो न जाता.

माधव आसमान की तरफ देखकर बोला, मानों देवताओं को अपनी निष्पापता का साक्षी बना रहा हो दुनिया का दस्तूर है, नहीं लोग बांभनों को हजारों रुपये क्यों दे देते हैं? कौन देखता है, परलोक में मिलता है या नहीं!

‘बड़े आदमियों के पास धन है, फ़ूंके. हमारे पास फूंकने को क्या है?’

‘लेकिन लोगों को जवाब क्या दोगे? लोग पूछेंगे नहीं, कफ़न कहां है?’

घीसू हंसा अबे, कह देंगे कि रुपये कमर से खिसक गये. बहुत ढूंढ़ा, मिले नहीं. लोगों को विश्वास न आएगा, लेकिन फिर वही रुपये देंगे.

माधव भी हंसा इस अनपेक्षित सौभाग्य पर. बोला बड़ी अच्छी थी बेचारी! मरी तो खूब खिला पिलाकर!

आधी बोतल से ज्यादा उड़ गयी. घीसू ने दो सेर पूडिय़ां मंगाई. चटनी, अचार, कलेजियां. शराबखाने के सामने ही दूकान थी. माधव लपककर दो पत्तलों में सारे सामान ले आया. पूरा डेढ़ रुपया खर्च हो गया. सिर्फ थोड़े से पैसे बच रहे.

दोनों इस वक्त इस शान में बैठे पूडिय़ां खा रहे थे जैसे जंगल में कोई शेर अपना शिकार उड़ा रहा हो. न जवाबदेही का खौफ था, न बदनामी की फ़िक्र. इन सब भावनाओं को उन्होंने बहुत पहले ही जीत लिया था.

घीसू दार्शनिक भाव से बोला हमारी आत्मा प्रसन्न हो रही है तो क्या उसे पुन्न न होगा?

माधव ने श्रद्धा से सिर झुकाकर तसदीक़ की जरूर से जरूर होगा. भगवान्, तुम अन्तर्यामी हो. उसे बैकुण्ठ ले जाना. हम दोनों हृदय से आशीर्वाद दे रहे हैं. आज जो भोजन मिला वह कभी उम्र भर न मिला था.

एक क्षण के बाद माधव के मन में एक शंका जागी. बोला क्यों दादा, हम लोग भी एक न एक दिन वहां जाएंगे ही?

घीसू ने इस भोले भाले सवाल का कुछ उत्तर न दिया. वह परलोक की बातें सोचकर इस आनन्द में बाधा न डालना चाहता था.

‘जो वहां हम लोगों से पूछे कि तुमने हमें कफ़न क्यों नहीं दिया तो क्या कहोगे?’

‘कहेंगे तुम्हारा सिर!’

‘पूछेगी तो जरूर!’

‘तू कैसे जानता है कि उसे कफ़न न मिलेगा? तू मुझे ऐसा गधा समझता है? साठ साल क्या दुनिया में घास खोदता रहा हूं? उसको कफ़न मिलेगा और बहुत अच्छा मिलेगा!’

माधव को विश्वास न आया. बोला कौन देगा? रुपये तो तुमने चट कर दिये. वह तो मुझसे पूछेगी. उसकी मांग में तो सेंदुर मैंने डाला था.

‘कौन देगा, बताते क्यों नहीं?’

‘वही लोग देंगे, जिन्होंने अबकी दिया. हां, अबकी रुपये हमारे हाथ न आएंगे.’

‘ज्यों ज्यों अंधेरा बढ़ता था और सितारों की चमक तेज होती थी, मधुशाला की रौनक भी बढ़ती जाती थी. कोई गाता था, कोई डींग मारता था, कोई अपने संगी के गले लिपटा जाता था. कोई अपने दोस्त के मुंह में कुल्हड़ लगाये देता था.’

वहां के वातावरण में सरूर था, हवा में नशा. कितने तो यहां आकर एक चुल्लू में मस्त हो जाते थे. शराब से ज्यादा यहां की हवा उन पर नशा करती थी. जीवन की बाधाएं यहां खींच लाती थीं और कुछ देर के लिए यह भूल जाते थे कि वे जीते हैं या मरते हैं. या न जीते हैं, न मरते हैं.

और यह दोनों बाप बेटे अब भी मजे ले लेकर चुसकियां ले रहे थे. सबकी निगाहें इनकी ओर जमी हुई थीं. दोनों कितने भाग्य के बली हैं! पूरी बोतल बीच में है.

भरपेट खाकर माधव ने बची हुई पूडिय़ों का पत्तल उठाकर एक भिखारी को दे दिया, जो खड़ा इनकी ओर भूखी आंखों से देख रहा था. और देने के गौरव, आनन्द और उल्लास का अपने जीवन में पहली बार अनुभव किया.

घीसू ने कहा ले जा, खूब खा और आशीर्वाद दे! जिसकी कमाई है, वह तो मर गयी. मगर तेरा आशीर्वाद उसे जरूर पहुंचेगा. रोयें रोयें से आशीर्वाद दो, बड़ी गाढ़ी कमाई के पैसे हैं!

माधव ने फिर आसमान की तरफ देखकर कहा वह बैकुण्ठ में जाएगी दादा, बैकुण्ठ की रानी बनेगी.

घीसू खड़ा हो गया और जैसे उल्लास की लहरों में तैरता हुआ बोला हां, बेटा बैकुण्ठ में जाएगी. किसी को सताया नहीं, किसी को दबाया नहीं. मरते मरते हमारी जिन्दगी की सबसे बड़ी लालसा पूरी कर गयी. वह न बैकुण्ठ जाएगी तो क्या ये मोटे मोटे लोग जाएंगे, जो गरीबों को दोनों हाथों से लूटते हैं, और अपने पाप को धोने के लिए गंगा में नहाते हैं और मन्दिरों में जल चढ़ाते हैं?

श्रद्धालुता का यह रंग तुरन्त ही बदल गया. अस्थिरता नशे की खासियत है. दु:ख और निराशा का दौरा हुआ.

माधव बोला मगर दादा, बेचारी ने जिन्दगी में बड़ा दु:ख भोगा. कितना दु:ख झेलकर मरी!

वह आंखों पर हाथ रखकर रोने लगा. चीखें मार मारकर.

घीसू ने समझाया क्यों रोता है बेटा, खुश हो कि वह माया जाल से मुक्त हो गयी, जंजाल से छूट गयी. बड़ी भाग्यवान थी, जो इतनी जल्द माया मोह के बन्धन तोड़ दिये.

और दोनों खड़े होकर गाने लगे """





In [ ]:
story = indic2en_model.translate_paragraph(ta_paragraph, 'hi', 'en')

story

100%|██████████| 244/244 [00:00<00:00, 10234.20it/s]
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


'At the door of the hut, both the father and the son were sitting quietly in front of a buried bonfire, and inside, the son \'s soldier, Bibi Budhia, was writhing in labour. There was such a heart - wrenching sound coming out of his mouth that both of them held the collage together. It was a winter night, nature was in a state of silence, the whole village was in darkness. There was a family of Chamars and the whole village was infamous. If Ghisu worked for one day, he would rest for three days. Madhav was so busy that if he worked for half an hour, he would drink for an hour. So they didnt take it. If there was even a handful of grain in the house, I swore to work for them. When it was two or four fissures away, Ghisu would climb up the tree and break the wood and Madhav would sell it from the market and as long as the money was there, both would roam around. There was no employment in the village. There was a village of farmers, there were fifty jobs for the hard working man. But he 

In [ ]:
import nltk

In [ ]:
refrence = """ At the door of the hut father and son sat silently by a burnt-out fire; inside, the son's young wife Budhiya lay in labor, writhing with pain. And from time to time such a heart-rending scream emerged from her lips that they both pressed their hands to their hearts. It was a winter night; everything was drowned in desolation. The whole village had been absorbed into the darkness.

Ghisu said, "It seems she won't live. She's been writhing in pain the whole day. Go on-- see how she is."

Madhav said in a pained tone, "If she's going to die, then why doesn't she go ahead and die? What's the use of going to see?"

"You’re pretty hard-hearted! You’ve enjoyed life with her for a whole year-- such faithlessness to her?"

"Well, I can't stand to see her writhing and thrashing around."

It was a family of Chamars, and notorious in the whole village. If Ghisu worked for one day, then he rested for three. Madhav was such a slacker that if he worked for an hour, then he smoked his chilam for an hour. Thus nobody hired them on. If there was even a handful of grain in the house, they both swore off working. When they'd fasted for a couple of days, then Ghisu climbed trees and broke off branches, and Madhav sold the wood in the market; and as long as that money lasted, they both spent their time wandering idly around. *When their hunger grew intense, they again broke off branches, or looked for some work.* There was no shortage of work in the village. It was a village of farmers; for a hard-working man there were fifty jobs. But people only sent for those two when they were forced to content themselves with getting out of two men the work of one.

If only the two had been ascetics, then they wouldn't have needed any exercises in self-discipline to achieve contentment and patience. This was their very nature. Theirs was a strange life. Except for two or three clay pots, they had no goods at all in the house. Covering their nakedness with torn rags, free from the cares of the world, laden with debt-- they suffered abuse, they suffered blows too, but not grief. They were so poor that without the smallest hope of repayment, people used to lend them something or other. When peas or potatoes were in season, they would dig up peas or potatoes from the fields and roast and eat them, or break off five or ten stalks of sugarcane and suck them at night. Ghisu had spent sixty years of his life in this pious manner, and Madhav, like a dutiful son, was following in his father's footsteps-- or rather, was making his name even more radiant.

*(1b)* This time too, both were seated by the fire, roasting potatoes that they had dug up from somebody's field. Ghisu's wife had passed away long ago. Madhav's marriage had taken place the year before. Since this woman had come, she had laid the foundations of civilization in the family. *Grinding grain, cutting grass, she arranged for a couple of pounds of flour,* and kept filling the stomachs of those two shameless ones. After she came, they both grew even more lazy and indolent; indeed, they even began to swagger a bit. If someone sent for them to work, then with splendid indifference they demanded double wages. That woman was dying today in childbirth. And these two were perhaps waiting for her to die, so they could sleep in peace.

Pulling out a potato and peeling it, Ghisu said, "Go see what shape she's in. We'll have the fuss over a ghost-witch-- what else! And here even the exorcist demands a rupee-- *from whose house would we get one?*"

Madhav suspected that if he went into the hut, Ghisu would finish off most of the potatoes. He said, "I'm afraid to go in."

"What are you afraid of? I'm here, after all."

"Then you go and see, all right?"

"When my wife died, for three days I never even left her side. And then, won't she be ashamed in front of me? I've never seen her face-- and today I should see her naked body? She won't even have bodily ease: if she sees me, she won't be able to thrash around freely."

"I'm thinking, if a child is born-- what then? Dried ginger, brown sugar, oil-- there's nothing at all in the house."

"Everything will come. If Bhagwan gives a child-- those people who now aren't giving a paisa, will send for us and give us things. I've had nine sons. There was never anything in the house, but this is how we managed every time."

*(1c)* A society in which those who labored night and day were not in much better shape than these two; a society in which compared to the peasants, those who knew how to exploit the peasants' weaknesses were much better off-- in such a society, the birth of this kind of mentality was no cause for surprise. We'll say that compared to the peasants, Ghisu was more insightful; and instead of joining the mindless group of peasants, he had joined the group of clever, scheming tricksters. Though indeed, he wasn't skilful in following the rules and customs of the tricksters. Thus while other members of his group became chiefs and headmen of villages, at him the whole village wagged its finger. But still, he did have the consolation that if he was in bad shape, at least he wasn't forced to do the back-breaking labor of the peasants, and others didn't take improper advantage of his simplicity and voicelessness.

Pulling out the potatoes, they both began to eat them burning hot. They had eaten nothing since the day before. They were too impatient to wait till the potatoes cooled. Both burned their tongues repeatedly. When the potatoes were peeled, their outer parts didn't seem so extremely hot. But the moment the teeth bit into them, the inner part burned the tongue and throat and roof of the mouth. Rather than keep that ember in the mouth, it was better to send it quickly along inward, where there was plenty of equipment for cooling it down. So they both swallowed very fast, although the attempt brought tears to their eyes.

*(1d)* Then Ghisu remembered a landowner's wedding procession, in which he had taken part twenty years before. The repletion that had been vouchsafed to him in that feast was a memorable event in his life, and even today its memory was fresh. He said, "I'll never forget that feast. Never since then have I had that kind of food, or such a full stomach. The girl's family fed puris to everyone. As much as they wanted! Great and small, everyone ate puris-- ones made with real ghi! Chutney, raita, three kinds of green vegetables, a flavorful stew, yoghurt, chutney, sweets. How can I tell you now what relish there was in that feast! There was no limit. Whatever thing you want, just ask! And however much you want, eat! People ate so much, ate so much, that nobody could even drink any water. And there the servers were-- setting hot, round, sweet-smelling pastries before you! You refuse, saying you don't want it. You push away the tray with your hand. But that's how they are-- they just keep on giving it. And when everybody had wiped their mouths, then everybody got a pan as well. But how could I be in any shape for a pan? I couldn't stand up. I just staggered off and lay down on my blanket. He had a heart as big as the ocean, that landowner!"

Enjoying the story of these grand festivities, Madhav said, "If only somebody would give us such a feast now!"

"As if anybody would feast anybody now! That was a different time. Now everybody thinks about economy-- 'don't spend money on weddings, don't spend money on religious festivals!'. Ask them-- what's this 'saving' of the poor people's wealth? There's no lack of 'saving'. But when it comes to spending, they think about economy!"

"You must have eaten twenty or so puris?"

"I ate more than twenty."

"I would have eaten up fifty."

"I couldn't have eaten less than fifty. I was hale and hearty. You're not half of what I was!"

After eating, they both drank some water, covered themselves with their dhotis, curled up, and went to sleep right there by the fire, as if two gigantic serpents lay coiled there.

And Budhiya was still moaning.

(2)

*(2a)* In the morning, when Madhav went into the hut and looked, his wife had grown cold. Flies were buzzing on her face. Her stony eyes had rolled upward. Her whole body was covered with dust. In her stomach, the baby had died.

Madhav came running to Ghisu. Then they both together began loudly lamenting and beating their breasts. When the neighbors heard the weeping and wailing, they came running. And following the ancient custom, they began to console the bereaved.

But this wasn't the occasion for an excessive show of grief. They had to worry about the shroud, and the wood. Money was as scarce in their house as meat in a raptor's nest.

Father and son went weeping to the village landlord. He hated the very sight of their faces. A number of times he had beaten them with his own hands-- for theft, or for not coming to work as they had promised. He asked, "What is it, Ghisua, why do you weep? Nowadays we don't even see you around. It seems that you no longer want to live in the village."

Ghisua fell prostrate on the ground, and said with tear-filled eyes, "Master, I'm in great trouble! Madhav's wife passed away last night. All day she was writhing in pain, Master; we two sat by her bed till midnight. Whatever medicines we could give her, we did. But she slipped away. Now we have no one to care for us, Master-- we're devastated-- our house is destroyed! I'm your slave. Now who but you will take care of her final rites? Whatever money we had at hand was used up on medicines. If the Master will show mercy, then she'll have the proper rites. To whose door should I come except yours?"

*(2b)* The Landlord Sahib was a compassionate man. But to show compassion to Ghisu was to try to dye a black blanket. He felt like saying, "Get out of here! *Keep the corpse in your house and let it rot!* Usually you don't come even when you're called-- now when you want something, you come and flatter me! You treacherous bastard! You villain!" But this was not the occasion for anger or revenge. Willingly or not, he pulled out two rupees and flung them down. But he didn't open his lips to say a single word of consolation. He didn't even look in Ghisu's direction-- as if he'd discharged a duty.

When the Landlord Sahib gave two rupees, then how could the village merchants and money-lenders have the nerve to refuse? Ghisu knew how to beat the drum of the landlord's name. One gave two paisas, another gave four paisas. In an hour, Ghisu had collected the sum of five rupees in ready cash. Someone gave grain, someone else gave wood. And in the afternoon Ghisu and Madhav went to the market to get a shroud. Meanwhile, people began to cut the bamboo poles, and so on.

The sensitive-hearted women of the village came and looked at the body. They shed a few tears at its helplessness, and went away.

(3)

*(3a)* When they reached the market, Ghisu said, "We've got enough wood to burn her, haven't we, Madhav?"

Madhav said, "Yes, there's plenty of wood. Now we need a shroud."

"So let's buy a light kind of shroud."

"Sure, what else! While the body is being carried along, night will come. At night, who sees a shroud?"

"What a bad custom it is that someone who didn't even get a rag to cover her body when she was alive, needs a new shroud when she's dead."

"After all, the shroud burns along with the body."

"What else is it good for? If we'd had these five rupees earlier, we would have given her some medicine."

Each of them inwardly guessed what the other was thinking. They kept wandering here and there in the market, until eventually evening came. [Sometimes they went to one cloth-seller's shop, sometimes to another. They looked at various kinds of fabric, they looked at silk and cotton, but nothing suited them.] The two arrived, by chance or deliberately, before a wine-house; and as if according to some prearranged decision, they went inside. For a little while they both stood there in a state of uncertainty. [Then Ghisu went to the counter and said, "Sir, please give us a bottle too."] *Ghisu bought one bottle of liquor, and some sesame sweets.* [After this some snacks came, fried fish came]. And they both sat down on the verandah and [peacefully] began to drink.

*(3b)* After drinking a number of cups in a row, both became elevated.

Ghisu said, "What's the use of wrapping her in a shroud? After all, it would only be burned. Nothing would go with her."

Looking toward the sky as if persuading the angels of his innocence, Madhav said, "It's the custom of the world-- why do these same people give thousands of rupees to the Brahmins? Who can tell whether a reward does or doesn't reach them in another world?"

"Rich people have wealth-- let them waste it! What do we have to waste?"

"But what will you tell people? Won't people ask where the shroud is?"

Ghisu laughed. "We'll say the money slipped out of my waistband-- we searched and searched for it, but it didn't turn up. [People won't believe it, but they'll still give the same sum again.]"

Madhav too laughed at this unexpected good fortune, *at defeating destiny in this way*. He said, "She was very good, the poor thing. Even as she died, she gave us a fine meal."

More than half the bottle had been finished. Ghisu ordered two sers of puris, a meat stew, and spiced liver and fried fish. There was a shop right next to the wine-house. Madhav ran over and brought everything back on two leaf-plates. The cost was fully one and a half rupees. Only a few paise were left.

*(3c)* Both then sat eating puris, with all the majesty of a tiger in the jungle pursuing his prey. They had no fear of being called to account, nor any concern about disgrace. They had passed through these stages of weakness long ago. Ghisu said in a philosophical manner, "If my soul is being pleased, then won't she receive religious merit?"

Madhav bowed his head in pious confirmation. "Certainly she'll certainly receive it. Bhagwan, you are the knower of hearts-- take her to Heaven! We're both giving her our heartfelt blessing. The feast I've had today-- I haven't had its equal in my whole life!"

After a moment a doubt arose in Madhav's heart. He said, "How about it-- we'll go there too someday, won't we?"

Ghisu gave no answer to this childish question. *He looked reproachfully at Madhav.* [He didn't want the thought of heavenly matters to interfere with this bliss.]

"When she asks us, there, why we didn't give her a shroud, what will you say?"

"Oh, shut up!"

"She'll certainly ask."

"How do you know that she won't get a shroud? Do you consider me such a donkey? I've lived in this world for sixty years-- and have I just been loitering around? She'll get a shroud, and [a very good one]-- *a much better than we would have given*."

Madhav was not convinced. He said, "Who will give it? You've gobbled up the rupees! [It's me she'll ask-- I'm the one who put the sindur in the parting of her hair.]"

Ghisu grew irritated. "I tell you, she'll get a shroud. Why don't you believe me?"

"Who will give the money-- why don't you tell me?"

"The same people will give it who gave it this time. But they won't put the rupees into our hands. *And if somehow we get our hands on them, we'll sit here and drink again just like this, and they'll give the shroud a third time.*"

*(3d)* As the darkness deepened and and the stars glittered more sharply, the tumult in the wine-house also increased. One person sang, another babbled, another embraced his companion, another pressed a glass to his friend's lips. Joy was in the atmosphere there. Intoxication was in the air. How many people become 'an ass with a glass'! *They came here only to taste the pleasure of self-forgetfulness.* More than liquor, the air here elevated their spirits. The disaster of life seized them and dragged them here. And for a while they forgot whether they were alive or dead-- or half-alive.

And these two, father and son, were still sipping with relish. Everyone's eyes had settled on them. How fortunate they were! They had a whole bottle between them.

After he had finished eating, Madhav picked up the leaf-plate of leftover puris and gave it to a beggar who was standing there looking at them with hungry eyes. And for the first time in his life he felt the pride and delight and thrill of giving.

Ghisu said, "Take it-- eat your fill, and give her your blessing. She whose earnings these are has died, but your blessing will certainly reach her. Bless her with every hair on your body-- these are the payment for very hard labor."

Madhav again looked toward the sky and said, "She'll go to Heaven-- she'll become the Queen of Heaven!"

*(3e)* Ghisu stood up, and as if swimming in waves of joy he said, "Yes, son, she'll go to Heaven! She never tormented anyone, she never oppressed anyone; even while dying, she fulfilled the greatest desire of our lives. If she doesn't go to Heaven, then will those fat rich people go-- who loot the poor with both hands, and go to the Ganges to wash away their sin, and offer holy water in temples?"

This mood of piety too changed; variability is the special quality of intoxication. It was the turn of despair and grief. Madhav said, "But the poor thing suffered a great deal in her life. Even her death was so painful!" Covering his eyes with his hands, he began to weep, [and sobbed loudly].

Ghisu consoled him: "Why do you weep, son? Be happy that she's been liberated from this net of illusion. She's escaped from the snare; she was very fortunate that she was able to break the bonds of worldly illusion so quickly."

And both, standing there, began to sing, "Temptress! Why do your eyes flash, temptress?"

The whole wine-house was absorbed in the spectacle, and these two drinkers, deep in intoxication, kept on singing. Then they both began to dance-- they leaped and jumped, fell down, flounced about, gesticulated, [strutted around]; and finally, overcome by drunkenness, they collapsed.
 """

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
 smoothie = SmoothingFunction().method4

In [ ]:
score = sentence_bleu([refrence], story, smoothing_function=smoothie)

In [ ]:
print("BLEU score for the story is :",score)

BLEU score for the story is : 0.6395698080914985


In [ ]:
import gensim
from gensim.models import Word2Vec

2021-10-01 11:00:05 | INFO | summarizer.preprocessing.cleaner | 'pattern' package not found; tag filters are not available for English


In [ ]:
data = []
for i in sent_tokenize(story):
    temp = []
    # tokenize the sentence into words
    for j in word_tokenize(i):
        temp.append(j.lower())
    data.append(temp)

In [ ]:
# skip gram model
model_1 = gensim.models.Word2Vec(data, min_count = 1,size = 100, window = 5, sg=0) 

# CBOW 
model_2 = gensim.models.Word2Vec(data, min_count = 1,size = 100, window = 5, sg=1) 

2021-10-01 11:40:06 | INFO | gensim.models.word2vec | collecting all words and their counts
2021-10-01 11:40:06 | INFO | gensim.models.word2vec | PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-01 11:40:06 | INFO | gensim.models.word2vec | collected 820 word types from a corpus of 3105 raw words and 246 sentences
2021-10-01 11:40:06 | INFO | gensim.models.word2vec | Loading a fresh vocabulary
2021-10-01 11:40:06 | INFO | gensim.models.word2vec | effective_min_count=1 retains 820 unique words (100% of original 820, drops 0)
2021-10-01 11:40:06 | INFO | gensim.models.word2vec | effective_min_count=1 leaves 3105 word corpus (100% of original 3105, drops 0)
2021-10-01 11:40:07 | INFO | gensim.models.word2vec | deleting the raw counts dictionary of 820 items
2021-10-01 11:40:07 | INFO | gensim.models.word2vec | sample=0.001 downsamples 59 most-common words
2021-10-01 11:40:07 | INFO | gensim.models.word2vec | downsampling leaves estimated 2021 word corpus (65.1% of

In [ ]:
print("Cosine similarity between money and drinking - Skip gram model is" , model_1.similarity("money","drinking"))

# while trying to find cosine similarity between few words I was facing the error which states "word" not in vocabulary while that word is surely in the story.
# How to deal with it? 

Cosine similarity between money and drinking - Skip gram model is -0.048346274


In [ ]:
print("Cosine similarity between money and drinking - CBOW model is" , model_2.similarity("money","drinking"))

Cosine similarity between money and drinking - CBOW model is 0.67639065


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
# EMBEDDINGS. 1st layer should be embediing

In [ ]:
while True:
  pass

In [ ]:
import nltk

In [ ]:
nltk.__version__

'3.2.5'